In [2]:
import ccxt
import time

# Replace with your actual Bybit API keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
TRADING_FEE = 0.001  # 0.1% trading fee per trade

def get_tradable_tokens():
    """Fetches all tradable tokens on Bybit and filters out low-volume pairs"""
    try:
        markets = bybit.load_markets()
        tradable_tokens = []
        for symbol in markets:
            if '/USDT' in symbol and ':USDT' not in symbol:
                market_data = bybit.fetch_ticker(symbol)
                volume = market_data.get('quoteVolume', 0)  # Get 24h trading volume
                if volume and volume >= MIN_VOLUME:
                    tradable_tokens.append(symbol)

        print(f"✅ Found {len(tradable_tokens)} tradable tokens")  # Debug print
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens: {e}")
        return []

def get_price(symbol):
    """Fetch latest price for a given token if the market exists"""
    try:
        if symbol in bybit.load_markets():
            ticker = bybit.fetch_ticker(symbol)
            
            if "last" in ticker:
                return float(ticker["last"])

        print(f"⚠️ Market not found: {symbol} (Skipping)")
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol}: {e}")
        return None

def check_intra_exchange_arbitrage(symbol):
    """Checks for spot vs futures arbitrage opportunities within Bybit"""
    spot_price = get_price(symbol)
    futures_symbol = symbol.replace('/USDT', '/USDT:USDT')

    if futures_symbol not in bybit.load_markets():
        print(f"⚠️ Futures market not found for {symbol} (Skipping)")
        return

    futures_price = get_price(futures_symbol)

    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price  # Calculate spread %
    net_profit = spread - (TRADING_FEE * 2)  # Adjust for fees

    if net_profit >= SPREAD_THRESHOLD:
        print(f"🔍 Intra-exchange Arbitrage Detected on Bybit for {symbol}")
        print(f"    ✅ Spot Price: {spot_price:.2f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.2f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print("-" * 50)

while True:
    try:
        bybit_tokens = get_tradable_tokens()
        if not bybit_tokens:
            print("❌ No tradable tokens found. Retrying in 10 seconds...")
            time.sleep(10)
            continue

        print("🔄 Checking for arbitrage opportunities...")

        for token in bybit_tokens:
            check_intra_exchange_arbitrage(token)

        print("✅ Completed this cycle. Waiting for next check...")
        time.sleep(CHECK_INTERVAL)
    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)  # Avoid infinite crash loops


❌ Error fetching tradable tokens: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ No tradable tokens found. Retrying in 10 seconds...
❌ Error fetching tradable tokens: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ No tradable tokens found. Retrying in 10 seconds...
❌ Error fetching tradable tokens: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ No tradable tokens found. Retrying in 10 seconds...
❌ Error fetching tradable tokens: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ No tradable tokens found. Retrying in 10 seconds...
❌ Error fetching tradable tokens: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ No tradable tokens found. Retrying in 10 seconds...
❌ Error fetching tradable tokens: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ No tradable tokens found. Retrying in 10 seconds...
❌ Error fetching tradable tokens: bybit GET https://api.bybit.com/v5/asset/coin/query-info?
❌ No tradable tokens found. Retr